In [106]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
using BenchmarkTools


In [175]:
nx = ny = nz = 70

nGrains = 10000


#Maximum sound speed in the model
c = 8000
#Frequency, hz
f0=1e6
#Period
t0 = 1.00 / f0
#Maximum spatial increment
dx = c/(8*f0)
#Maximum time increment
dt = dx/(c*sqrt(3))

baseMat = Main.EFIT.AnisoMats["X6CrNi1811"]
c = baseMat.c
ρ = baseMat.ρ
materials = Array{Main.EFIT.AnisoMat,1}()

Main.EFIT.AnisoMat[]

In [176]:
function initFloodFill!(nx::Number,ny::Number,nz::Number, nGrains::Number, matGrid::Array{Int,3}, positions::Array{Int,2})
    Qs = Array{Array{CartesianIndex}}(undef, nGrains)
    offsets = [CartesianIndex(-1,0,0),CartesianIndex(1,0,0),CartesianIndex(0,-1,0),CartesianIndex(0,1,0),CartesianIndex(0,0,-1),CartesianIndex(0,0,1)]
    #Initialize the queue
    for i in 1:size(positions,1)
        p = CartesianIndex(positions[i,1],positions[i,2],positions[i,3])
        Qs[i] = Array{CartesianIndex,1}()
        matGrid[p]=i
        for o in offsets
            new = p + o
            if new[1] < nx && new[1] > 0 && new[2] < ny && new[2] > 0 && new[3] < nz && new[3] > 0
                if matGrid[new] == 0
                    push!(Qs[i], new)
                end
            end
        end
    end
    return Qs
end

initFloodFill! (generic function with 1 method)

In [186]:
const offsets = [CartesianIndex(-1,0,0),CartesianIndex(1,0,0),CartesianIndex(0,-1,0),CartesianIndex(0,1,0),CartesianIndex(0,0,-1),CartesianIndex(0,0,1)]

function stepFloodFill!(matGrid::Array{Int,3}, Qs::Array{Array{CartesianIndex}})
    length::Int = 0
    
    for i in 1:size(Qs,1)
        if size(Qs[i],1) > 0
            q0 = popfirst!(Qs[i])
            if matGrid[q0] == 0
                matGrid[q0] = i
                
                for o in offsets
                    new = q0 + o
                    if new[1] <= nx && new[1] > 0 && new[2] <= ny && new[2] > 0 && new[3] <= nz && new[3] > 0
                        if matGrid[new] == 0
                            pushfirst!(Qs[i], new)
                            length += 1 
                        end
                    end
                end
            end
        end
    end
    return length
    
end

stepFloodFill! (generic function with 2 methods)

In [181]:
#matGrid = zeros(Int,nx,ny,nz)
cmap=:curl
fig = Figure()
#ax = LScene(fig[1,1])
fig[1,1] = Axis(fig[1,1])
plt = heatmap!(fig[1,1], matGrid[floor(Int,end/2), :,:],colormap=cmap)
#plt = volume!(fig[1,1], matGrid, colormap=cmap,colorrange=(0,nGrains*2))
#plt = volume!(fig[1,1], matGrid, algorithm = :iso, isorange = 1000, isovalue = 1000)
display(fig)

GLMakie.Screen(...)

In [182]:
matGrid = zeros(Int,nx,ny,nz)
positions = reduce(hcat,[rand(1:nx, nGrains),rand(1:ny, nGrains),rand(1:nz, nGrains)]);
Qs = initFloodFill!(nx,ny,nz,nGrains,matGrid,positions);


In [188]:
count = 1
while count > 0
    count = stepFloodFill!(matGrid, Qs)
    plt[1] = matGrid[floor(Int,end/2), :,:];
    println(count)
end    

40955
36040
39932
31344
26283
25281
24243
21383
21340
7921
23889
21300
23199
16138
8454
17737
16895
17873
13405
8039
10814
5053
8155
11422
7479
9864
6239
8262
10412
6105
6785
7383
8536
11163
10607
9409
8557
8232
6945
7653
7366
6869
6205
6875
6582
6184
5560
5389
5886
6055
5256
5397
5206
5913
5664
4985
4983
5007
4300
4210
4333
4490
4051
3797
3991
3944
3456
3574
3416
3351
3423
3187
2950
3002
2743
2576
2508
2465
2373
2307
2112
2128
1938
1733
1676
1650
1494
1449
1316
1355
1270
1069
1074
929
901
764
789
663
598
556
541
503
453
439
409
367
320
315
264
280
242
185
180
151
141
101
92
100
94
88
56
53
65
49
47
52
38
17
38
40
22
11
21
10
20
13
15
15
17
14
6
7
1
2
1
1
0
